In [0]:
# !pip install dotenv

In [0]:
%pip install -r ../requirements.txt

In [0]:
%restart_python

In [0]:
dbutils.library.restartPython()

In [0]:
dbutils.widgets.dropdown('env', 'dev', ['dev', 'staging', 'prod'], 'Environment Name')

In [0]:
from booking.utils.notebook_utils import load_and_set_env_vars, load_config

In [0]:
from booking.common import MLflowTrackingConfig, FeatureStoreTableConfig, LabelsTableConfig
from booking.model_train import ModelTrain, ModelTrainConfig
from booking.utils.logger_utils import get_logger

In [0]:
_logger = get_logger()

In [0]:
# Set pipeline name
pipeline_name = 'model_train'

In [0]:

# Load pipeline config yaml file (../conf/pipeline_configs/{pipeline_name}.yml)
pipeline_config = load_config(pipeline_name)

In [0]:
# Params passed via ../conf/{env}/.{env}.env and ../conf/.base_data_
# params
env_vars = load_and_set_env_vars(env=dbutils.widgets.get('env'))

In [0]:
mlflow_tracking_cfg = MLflowTrackingConfig(run_name=pipeline_config['mlflow_params']['run_name'],
                                           experiment_path=env_vars['model_train_experiment_path'],
                                           model_name=env_vars['model_name'])

In [0]:
feature_store_table_cfg = FeatureStoreTableConfig(database_name=env_vars['feature_store_database_name'],
                                                  table_name=env_vars['feature_store_table_name'],
                                                  primary_keys=env_vars['feature_store_table_primary_keys'])

In [0]:
# Set LabelsTableConfig
labels_table_cfg = LabelsTableConfig(database_name=env_vars['labels_table_database_name'],
                                 table_name=env_vars['labels_table_name'],
                                 label_col=env_vars['labels_table_label_col'])

In [0]:
pipeline_params = pipeline_config['pipeline_params']

In [0]:
model_params = pipeline_config['model_params']

In [0]:
cfg = ModelTrainConfig(mlflow_tracking_cfg=mlflow_tracking_cfg,
                       feature_store_table_cfg=feature_store_table_cfg,
                       labels_table_cfg=labels_table_cfg,
                       pipeline_params=pipeline_params,
                       model_params=model_params,
                       conf=pipeline_config,    # Track pipeline_config to mlflow
                       env_vars=env_vars        # Track env_vars to mlflow
                      )

In [0]:
model_train_pipeline = ModelTrain(cfg)
model_train_pipeline.run()